In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import linregress
from sklearn import linear_model
import math

# Estimate yield for all sites with reduced phenology

In [121]:
#Merge DSSAT generated weather variables
def split_csv(csvFilePath, newDirPath,splitKey,sce_n):
    df = pd.read_csv(csvFilePath)
    df = df.dropna()
    df = df.drop_duplicates()
    for (n), group in df.groupby([splitKey]):
        group['cul']=sce_n
        group['year']=group.year #np.arange(group.shape[0])
        group['yearn']=np.arange(group.shape[0])
        group.rename(columns={'cul':'sce'},inplace=True)
        outfile=f'{newDirPath}{n[2:]}.csv'
        if os.path.exists(outfile):
            group.to_csv(outfile,mode='a',header=False,index=False)
        else:
            group.to_csv(outfile,index=False)
            
def processwea(nursery,workdir,datadir): #loclist
    sce=["arc_hist"]+[x+"_"+y for x in ["gfdl","ipsl","mkes","mpi","mri"] for y in ["hist","ssp126","ssp370","ssp585"]]
    sce_name=["ar_h"]+[x+"_"+y for x in ["gf","ip","mk","mp","mr"] for y in ["h","1","3","5"]]
    split_place=[4 if nursery in ["IWWYT_IRR","IWWYT_SA"] else 3][0]
    os.chdir(workdir)
    if not os.path.exists(nursery): os.makedirs(nursery)
    filelist=os.listdir(datadir+"/"+nursery+"_arc_hist") #keep site are same across scenarios
    loclist=[x.split('_')[split_place] for x in filelist] #site list
    #loclist=['41108']
    for loc in range(len(loclist)): #create a a folder named as loc
        if not os.path.exists(nursery+"/"+loclist[loc]): os.makedirs(nursery+"/"+loclist[loc]) #creat the site directory if it doesn't exist
        for s in range(len(sce)): #scenarios
            #make sure site are same across scenarios
            wea_name=filelist[loc]
            if s==0: 
                weafile=wea_name
            else:
                weafile=wea_name.replace('arc_hist',sce[s])
            weafile=datadir+"/"+nursery+"_"+sce[s]+"/"+weafile #the weather file name
            #split weather file into 
            if os.path.exists(weafile):
                split_csv(csvFilePath=weafile,
                          newDirPath=workdir+"/"+nursery+"/"+loclist[loc]+"/",
                          splitKey='cul',sce_n=sce_name[s])
nursery="IWWYT_SA"
#workdir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/temp"
workdir="E:/Works/temp"
datadir="H:/NCC_result_IWIN_Phe_Cli4all/DSSATForecast4AllSites"
processwea(nursery,workdir,datadir)

In [29]:
def predictyld(nursery,workdir,datadir): #loclist
    rf=RandomForestRegressor(n_estimators=1000,random_state=42)
    split_place=[4 if nursery in ["IWWYT_IRR","IWWYT_SA"] else 3][0]
    cli=[x+'_'+y for y in ['veg','rep','gfi'] for x in ['tavg','tdr','gdd30','rs','p','rh','vpd','ws']]
    cli_all=sum([[x+'_sqr',x] for x in cli],[])
    #Genotyp file
    gen=pd.read_csv(datadir+"/"+nursery+"/"+nursery+"_gen.csv")
    gen=gen[['genid','gen']]
    #trial file
    trial=pd.read_csv(datadir+"/"+nursery+"/Final_SimFilledSow__"+nursery+".csv")
    trial['id']=trial.index
    trial['gen']=[x.split('@')[1] for x in trial.sid_gen]
    trial=trial[['id','gen','loc','wea_file','year','yearofsow','dayofsow','dayofhar','yld','head','anth','matu','sim_head','sim_anth','sim_matu']]
    matu=trial[['loc','matu']].groupby('loc').mean().dropna().reset_index()
    grand_mean=trial.yld.mean()
    Gi=trial[['gen','yld']].groupby('gen').mean().dropna().reset_index()
    Gi['Gi']=Gi['yld']-Gi['yld'].mean()
    gen=gen.merge(Gi[['gen','Gi']],how='left')
    #File to retrieve relationship between maturity and yield
    trial=pd.read_csv(datadir+"/"+nursery+"/"+nursery+"_Obs_Sim_Yld_Phe_Climate.csv")
    Ej=trial[['loc','yld']].groupby('loc').mean().dropna().reset_index()
    Ej=Ej.merge(matu,on='loc',how='left')
    Ej_=Ej.dropna()
    #Robust linear regression
    ransac=linear_model.RANSACRegressor()
    ransac.fit(Ej_[['matu']],Ej_[['yld']])
    E_fit=ransac.estimator_.intercept_[0]+ransac.estimator_.coef_[0]*Ej['matu']
    Ej['Ej']=Ej['yld']-E_fit-E_fit.mean()

    #Replace old Ej by Ej that addresses the large environment variations
    trial.drop('Ej', inplace=True, axis=1)
    trial=trial.merge(Ej[['loc','Ej']],on='loc',how='left')
    trial['delYld1']=trial['yld']-grand_mean-trial['Gi']-trial['Ej']

    gen.to_csv(workdir+"/"+nursery+"/"+nursery+"_Gi_grand-mean_"+str(grand_mean)+".csv",index=False)
    Ej[['loc','Ej']].to_csv(workdir+"/"+nursery+"/"+nursery+"_Ej.csv",index=False)
    #bunk totalbunk
    #Identify the genotypes
    gid=gen.genid.tolist() #genotype list
    #perchunk=math.ceil(len(gid)/int(chunks))
    #gid=gid[(int(n_chunks)-1)*perchunk:int(n_chunks)*perchunk]
    loclist=os.listdir(workdir+"/"+nursery)
    for g in gid:
        Yld=trial[trial.gen==gen.loc[g,'gen']][['delYld1','Gi','Ej']+cli].dropna()
        if Yld.shape[0]>=5: #make sure there are at least 10 sites
            for c in cli: Yld[c+'_sqr']=Yld[c]**2
            y=Yld['delYld1']
            X=Yld[cli_all]
            rf.fit(X,y)
            for loc in range(len(loclist)):
                weafile=workdir+"/"+nursery+"/"+loclist[loc]+"/"+str(g).zfill(4)+".csv"
                if os.path.exists(weafile):
                    wea=pd.read_csv(weafile)
                    wea=wea.dropna()
                    wea=wea.drop_duplicates()
                    for c in cli:wea[c+'_sqr']=wea[c]**2
                    wea["yld"]=rf.predict(wea[cli_all])
                    wea=wea.pivot(index="sce",columns="yearn",values="yld").reset_index()
                    #wea.to_csv(workdir+"/"+nursery+"/"+loclist[loc]+"/yld_g"+str(g).zfill(4)+".csv",index=False)
                    wea.to_csv(workdir+"/"+nursery+"/"+loclist[loc]+"/"+str(g).zfill(4)+".csv",index=False)
datadir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/Input"  
nursery="IWWYT_SA"
#workdir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/temp"
workdir="E:/Works/temp"
predictyld(nursery,workdir,datadir)

# Estimate yield for all sites with trend removed phenology

# Estimate yield for all sites with reduced phenology but yield elevation by higher CO2 concentration

In [196]:
#Merge DSSAT generated weather variables
def split_csv(csvFilePath, newDirPath,splitKey,sce_n):
    df = pd.read_csv(csvFilePath)
    df = df.dropna()
    df = df.drop_duplicates()
    for (n), group in df.groupby([splitKey]):
        group['cul']=sce_n
        group['year']=group.year #np.arange(group.shape[0])
        group['yearn']=np.arange(group.shape[0])
        group.rename(columns={'cul':'sce'},inplace=True)
        outfile=f'{newDirPath}{n[2:]}.csv'
        if os.path.exists(outfile):
            group.to_csv(outfile,mode='a',header=False,index=False)
        else:
            group.to_csv(outfile,index=False)
            
def processwea(nursery,workdir,datadir): #loclist
    sce=["arc_hist"]+[x+"_"+y for x in ["gfdl","ipsl","mkes","mpi","mri"] for y in ["hist","ssp126","ssp370","ssp585"]]
    sce_name=["ar_h"]+[x+"_"+y for x in ["gf","ip","mk","mp","mr"] for y in ["h","1","3","5"]]
    split_place=[4 if nursery in ["IWWYT_IRR","IWWYT_SA"] else 3][0]
    os.chdir(workdir)
    if not os.path.exists(nursery): os.makedirs(nursery)
    filelist=os.listdir(datadir+"/"+nursery+"_arc_hist") #keep site are same across scenarios
    loclist=[x.split('_')[split_place] for x in filelist] #site list
    #loclist=['41108']
    for loc in range(len(loclist)): #create a a folder named as loc
        if not os.path.exists(nursery+"/"+loclist[loc]): os.makedirs(nursery+"/"+loclist[loc]) #creat the site directory if it doesn't exist
        for s in range(len(sce)): #scenarios
            #make sure site are same across scenarios
            wea_name=filelist[loc]
            if s==0: 
                weafile=wea_name
            else:
                weafile=wea_name.replace('arc_hist',sce[s])
            weafile=datadir+"/"+nursery+"_"+sce[s]+"/"+weafile #the weather file name
            #split weather file into 
            if os.path.exists(weafile):
                split_csv(csvFilePath=weafile,
                          newDirPath=workdir+"/"+nursery+"/"+loclist[loc]+"/",
                          splitKey='cul',sce_n=sce_name[s])
nursery="IWWYT_SA"
#workdir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/temp"
workdir="E:/Works/temp"
datadir="H:/NCC_result_IWIN_Phe_Cli4all/DSSATForecast4AllSites"
processwea(nursery,workdir,datadir)

In [197]:
#yield promotion rate, 0.068% per 1 ppm CO2, corresponding to the mean CO2 conentration between 1981-2010
#IWWYT_IRR and IWWYR_SA's CO2 concentration is not right, need read from other file - CO2_concentration.csv
def predictyld(nursery,workdir,datadir): #loclist
    rf=RandomForestRegressor(n_estimators=1000,random_state=42)
    split_place=[4 if nursery in ["IWWYT_IRR","IWWYT_SA"] else 3][0]
    cli=[x+'_'+y for y in ['veg','rep','gfi'] for x in ['tavg','tdr','gdd30','rs','p','rh','vpd','ws']]
    cli_all=sum([[x+'_sqr',x] for x in cli],[])
    #Genotyp file
    gen=pd.read_csv(datadir+"/"+nursery+"/"+nursery+"_gen.csv")
    gen=gen[['genid','gen']]
    #trial file
    trial=pd.read_csv(datadir+"/"+nursery+"/Final_SimFilledSow__"+nursery+".csv")
    trial['id']=trial.index
    trial['gen']=[x.split('@')[1] for x in trial.sid_gen]
    trial=trial[['id','gen','loc','wea_file','year','yearofsow','dayofsow','dayofhar','yld','head','anth','matu','sim_head','sim_anth','sim_matu']]
    matu=trial[['loc','matu']].groupby('loc').mean().dropna().reset_index()
    grand_mean=trial.yld.mean()
    Gi=trial[['gen','yld']].groupby('gen').mean().dropna().reset_index()
    Gi['Gi']=Gi['yld']-Gi['yld'].mean()
    gen=gen.merge(Gi[['gen','Gi']],how='left')
    #File to retrieve relationship between maturity and yield
    trial=pd.read_csv(datadir+"/"+nursery+"/"+nursery+"_Obs_Sim_Yld_Phe_Climate.csv")
    Ej=trial[['loc','yld']].groupby('loc').mean().dropna().reset_index()
    Ej=Ej.merge(matu,on='loc',how='left')
    Ej_=Ej.dropna()
    #Robust linear regression
    ransac=linear_model.RANSACRegressor()
    ransac.fit(Ej_[['matu']],Ej_[['yld']])
    E_fit=ransac.estimator_.intercept_[0]+ransac.estimator_.coef_[0]*Ej['matu']
    Ej['Ej']=Ej['yld']-E_fit-E_fit.mean()

    #Replace old Ej by Ej that addresses the large environment variations
    trial.drop('Ej', inplace=True, axis=1)
    trial=trial.merge(Ej[['loc','Ej']],on='loc',how='left')
    trial['delYld1']=trial['yld']-grand_mean-trial['Gi']-trial['Ej']

    gen.to_csv(workdir+"/"+nursery+"/"+nursery+"_Gi_grand-mean_"+str(grand_mean)+".csv",index=False)
    Ej[['loc','Ej']].to_csv(workdir+"/"+nursery+"/"+nursery+"_Ej.csv",index=False)
      
    #bunk totalbunk
    #Identify the genotypes
    gid=gen.genid.tolist() #genotype list
    #perchunk=math.ceil(len(gid)/int(chunks))
    #gid=gid[(int(n_chunks)-1)*perchunk:int(n_chunks)*perchunk]
    loclist=os.listdir(workdir+"/"+nursery)
    for g in gid:
        Yld=trial[trial.gen==gen.loc[g,'gen']][['delYld1','Gi','Ej']+cli].dropna()
        if Yld.shape[0]>=5: #make sure there are at least 10 sites
            for c in cli: Yld[c+'_sqr']=Yld[c]**2
            y=Yld['delYld1']
            X=Yld[cli_all]
            rf.fit(X,y)
            for loc in range(len(loclist)):
                if os.path.isdir(workdir+"/"+nursery+"/"+loclist[loc]):
                    weafile=workdir+"/"+nursery+"/"+loclist[loc]+"/"+str(g).zfill(4)+".csv"
                    if os.path.exists(weafile):
                        wea=pd.read_csv(weafile)
                        wea=wea.dropna()
                        wea=wea.drop_duplicates()
                        for c in cli: wea[c+'_sqr']=wea[c]**2
                        wea["yld"]=rf.predict(wea[cli_all])
                        ################Add delta yield to simulated yield caused by elevated CO2########################
                        #Observed yield of loc mean yield to compute yield elevation due to CO2
                        if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
                            loc_yld=grand_mean #.mean()
                        else:
                            loc_yld=np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values)
                        #Estimate delta yield
                        temp=pd.DataFrame({'sce':wea.sce.str[3:],'year':wea.year})
                        co2=pd.read_csv(datadir+"/"+"CO2_ChangeInSSP.csv")
                        temp=temp.merge(co2,how='left')
                        temp['co2_delyld']=loc_yld*0.00068*temp.co2_del #0.068% per 1 ppm CO2
                        wea["yld"]=wea["yld"]+temp["co2_delyld"]
                        #################################################################################################
                        wea=pd.pivot_table(data=wea,index="sce",columns="yearn",values="yld",aggfunc='mean').reset_index()
                        #wea.to_csv(workdir+"/"+nursery+"/"+loclist[loc]+"/yld_g"+str(g).zfill(4)+".csv",index=False)
                        wea.to_csv(workdir+"/"+nursery+"/"+loclist[loc]+"/"+str(g).zfill(4)+".csv",index=False)
datadir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/Input"  
nursery="IWWYT_SA"
#workdir="H:/NCC_result_IWIN_Phe_Cli4all/HPC/temp"
workdir="E:/Works/temp"
predictyld(nursery,workdir,datadir)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

...

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py:64: RuntimeWarning: Mean of empty slice
  if np.nanmean(Ej.loc[Ej['loc']==loclist[0],'yld'].values).astype('str')=='nan':
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27572\3717957057.py: